In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#
# usage: ./increase_picture.py hogehoge.jpg
#

import cv2
import numpy as np
import sys
import os
import glob

# ヒストグラム均一化
def equalizeHistRGB(src):
    
    RGB = cv2.split(src)
    Blue   = RGB[0]
    Green = RGB[1]
    Red    = RGB[2]
    for i in range(3):
        cv2.equalizeHist(RGB[i])

    img_hist = cv2.merge([RGB[0],RGB[1], RGB[2]])
    return img_hist

# ガウシアンノイズ
def addGaussianNoise(src):
    row,col,ch= src.shape
    mean = 0
    var = 0.1
    sigma = 15
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    gauss = gauss.reshape(row,col,ch)
    noisy = src + gauss
    
    return noisy

# salt&pepperノイズ
def addSaltPepperNoise(src):
    row,col,ch = src.shape
    s_vs_p = 0.5
    amount = 0.004
    out = src.copy()
    # Salt mode
    num_salt = np.ceil(amount * src.size * s_vs_p)
    coords = [np.random.randint(0, i-1 , int(num_salt))
                 for i in src.shape]
    out[coords[:-1]] = (255,255,255)

    # Pepper mode
    num_pepper = np.ceil(amount* src.size * (1. - s_vs_p))
    coords = [np.random.randint(0, i-1 , int(num_pepper))
             for i in src.shape]
    out[coords[:-1]] = (0,0,0)
    return out

#コントラスト調整
def pitchContrast():
    # ルックアップテーブルの生成
    min_table = 50
    max_table = 205
    diff_table = max_table - min_table
    gamma1 = 0.75
    gamma2 = 1.5
    
    LUT_HC = np.arange(256, dtype = 'uint8' )
    LUT_LC = np.arange(256, dtype = 'uint8' )
    LUT_G1 = np.arange(256, dtype = 'uint8' )
    LUT_G2 = np.arange(256, dtype = 'uint8' )
    LUTs = []
    
        # ハイコントラストLUT作成
    for i in range(0, min_table):
        LUT_HC[i] = 0
               
    for i in range(min_table, max_table):
        LUT_HC[i] = 255 * (i - min_table) / diff_table
                                  
    for i in range(max_table, 255):
        LUT_HC[i] = 255

    # その他LUT作成
    for i in range(256):
        LUT_LC[i] = min_table + i * (diff_table) / 255
        LUT_G1[i] = 255 * pow(float(i) / 255, 1.0 / gamma1) 
        LUT_G2[i] = 255 * pow(float(i) / 255, 1.0 / gamma2)

    LUTs.append(LUT_HC)
    LUTs.append(LUT_LC)
    LUTs.append(LUT_G1)
    LUTs.append(LUT_G2)
    return LUTs


def doMizumashi(in_dir_name):
    
    # 保存
    if not os.path.isdir("trans_images\\{}".format(os.path.split(in_dir_name)[1])):
        os.makedirs("trans_images\\{}".format(os.path.split(in_dir_name)[1]))   
    
    # 平滑化用
    average_square = (10,10)
    gyudon_img = glob.glob('{}\*.jpg'.format(in_dir_name))
    
    # 画像の読み込み
    for file_name in gyudon_img:
        img_src = cv2.imread(file_name)
        trans_img = []
        trans_img.append(img_src)
    
    # LUT変換
        LUTs = pitchContrast()
        for i, LUT in enumerate(LUTs):
               trans_img.append( cv2.LUT(img_src, LUT))

        # 平滑化      
        trans_img.append(cv2.blur(img_src, average_square))      

    # ヒストグラム均一化
        trans_img.append(equalizeHistRGB(img_src))

    # ノイズ付加
        trans_img.append(addGaussianNoise(img_src))
        trans_img.append(addSaltPepperNoise(img_src))

    # 反転
        flip_img = []
        for img in trans_img:
            flip_img.append(cv2.flip(img, 1))
        trans_img.extend(flip_img)

    
        base =  os.path.splitext(os.path.basename(file_name))[0] + "_"
        img_src.astype(np.float64)
        for i, img in enumerate(trans_img):
            # 比較用
            # cv2.imwrite("trans_images/" + base + str(i) + ".jpg" ,cv2.hconcat([img_src.astype(np.float64), img.astype(np.float64)]))
            cv2.imwrite("trans_images\\{}\\{}{}".format(os.path.split(in_dir_name)[1],base,str(i)) + ".jpg" ,img) 
        
doMizumashi('data\\img_crop\\gyudon')

C:\Users\ilrma\anaconda3\lib\site-packages\ipykernel_launcher.py:48: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
C:\Users\ilrma\anaconda3\lib\site-packages\ipykernel_launcher.py:54: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


In [32]:
in_dir_name = 'data\\img_crop\\gyudon'
file_name = 'data\\img_crop\\gyudon\\0000001.jpg'
base =  os.path.splitext(os.path.basename(file_name))[0] + "_"
i=1
print(base)
print("trans_images\\{}\\{}{}".format(os.path.split(in_dir_name)[1],base,str(i)) + ".jpg")
print("trans_images\\{}".format(os.path.split(in_dir_name)[1]))

0000001_
trans_images\gyudon\0000001_1.jpg
trans_images\gyudon


In [51]:
gyudon_img = glob.glob('{}\*.jpg'.format('data\\original\\chi-zugyudon'))
i=1
for n in gyudon_img:
    os.rename(n,'data\\original\\chi-zugyudon\\{:0>5}.jpg'.format(str(i)))
    i += 1